In [3]:
import linecache
import os
import time
import random
import matplotlib.pyplot as plt
import pandas as pd
import sys
import threading
import time
import resource
resource.setrlimit(resource.RLIMIT_NOFILE, (65536, 65536))

In [4]:
import mariadb
import sys
import linecache
import os
import time
import random
import matplotlib.pyplot as plt
import pandas as pd
import sys
import threading
import time
import docker

In [5]:
# 每个单独的op
class Operation:
    op_type = True  #true is write
    variable = 0
    value = 0
    
    def __init__(self, op_type, variable, value):
        self.op_type = op_type
        self.variable = variable
        self.value = value

    def Read(self,variable):
        self.op_type = False
        self.variable = variable
        self.value = 0
    def Write(self,variable,value):
        self.op_type = True
        self.variable = variable
        self.value = value
    def Display_info(self):
        if(op_type==True):
            print("write," + str(variable) + "," + str(value))
        elif(op_type==False):
            print("read," + str(variable) + "," + str(value))
        else:
            print("Error in Operation op_type!")

In [6]:
import numpy as np
from matplotlib import pyplot as plt

def Zipf(a: np.float64, min: np.uint64, max: np.uint64, size=None):
    """
    Generate Zipf-like random variables,
    but in inclusive [min...max] interval
    """
    if min == 0:
        raise ZeroDivisionError("")

    v = np.arange(min, max+1) # values to sample
    p = 1.0 / np.power(v, a)  # probabilities
    p /= np.sum(p)            # normalized

    return np.random.choice(v, size=size, replace=True, p=p)


In [7]:
def set_initval(counter,variable_num):
    seq = []
    for i in range (0,variable_num):
        seq.append(i)
        i = i + 1
    counter = counter.fromkeys(seq, 0)
    return counter

In [8]:
# 用法：randon_pick([true,false],[0.5,0.5])
def random_pick(some_list, probabilities): 
    x = random.uniform(0,1) 
    cumulative_probability = 0.0 
    for item, item_probability in zip(some_list, probabilities): 
        cumulative_probability += item_probability 
        if x < cumulative_probability:
               break 
    return item 

In [9]:
def uniform_generator(output_path, his, client, trans, ops, var, wr):
    
    #define params 
    his_file = his 
    client_num = client
    trans_num = trans
    op_num = ops
    variable_num = var 
    all_hist = [] 
    for h in range (0,his_file): 
        doc =open(output_path+"hist_"+str(h)+".txt",'w')
        new_hist = [] 
        counter = {} 
        counter = set_initval(counter,variable_num) 
        for c in range (0,client_num):
            new_client = []
            for t in range (0,trans_num):
                new_transaction = []
                for op in range (0,op_num):
                    #按照比例选and是读或写
                    if wr == 55:
                        op_type = random_pick([True,False],[0.5,0.5])
                    elif wr == 19:
                        op_type = random_pick([True,False],[0.1,0.9])
                    elif wr ==91:
                        op_type = random_pick([True,False],[0.9,0.1])
                    else:
                        print('Wrong input wr!')

                    if(op_type==False): #READ
                    #   随机选择variable编号
                        variable = random.randint(0,variable_num-1)
                        new_op = Operation(False,variable,0)
                    elif(op_type==True):
                        variable = random.randint(0,variable_num-1)
                        value = counter[variable] +1
                        counter[variable] = value #更新counter内的值
                        new_op = Operation(True,variable,value)
                    else:
                        print("Error in op_type!")
                    #new_op.Display_info()
                    
                    if(new_op.op_type==True):
                        doc.write("write," + str(new_op.variable) + "," + str(new_op.value)+"\n")
                    elif(new_op.op_type==False):
                        doc.write("read," + str(new_op.variable) + "," + str(new_op.value)+"\n")
                    else:
                        print("Error in file Writing!")
        doc.close()
        print(output_path+"hist_"+str(h)+".txt"+" succeeded.")


In [10]:
def zipf_generator(output_path, his, client, trans, ops, var, wr):
    
    #define params 
    his_file = his 
    client_num = client
    trans_num = trans
    op_num = ops
    variable_num = var 
    all_hist = [] 
    for h in range (0,his_file): 
        doc =open(output_path+"hist_"+str(h)+".txt",'w')
        new_hist = [] 
        counter = {} 
        counter = set_initval(counter,variable_num)
        
        min = np.uint64(1)
        max = np.uint64(var)

        q = Zipf(1, min, max, client_num*trans_num*op_num)
        variable_list = [int(x)-1 for x in q]
        counter_for_variable = 0
        
        for c in range (0,client_num):
            new_client = []
            for t in range (0,trans_num):
                new_transaction = []
                for op in range (0,op_num):
                    #按照比例选and是读或写
                    if wr == 55:
                        op_type = random_pick([True,False],[0.5,0.5])
                    elif wr == 19:
                        op_type = random_pick([True,False],[0.1,0.9])
                    elif wr ==91:
                        op_type = random_pick([True,False],[0.9,0.1])
                    else:
                        print('Wrong input wr!')

                    if(op_type==False): #READ
                    #   使用zipf分布选择variable编号
                        variable = variable_list[counter_for_variable]
                        counter_for_variable = counter_for_variable+1
                        new_op = Operation(False,variable,0)
                    elif(op_type==True):
                        variable = variable_list[counter_for_variable]
                        counter_for_variable = counter_for_variable+1
                        value = counter[variable] +1
                        counter[variable] = value #更新counter内的值
                        new_op = Operation(True,variable,value)
                    else:
                        print("Error in op_type!")
                    #new_op.Display_info()
                    
                    if(new_op.op_type==True):
                        doc.write("write," + str(new_op.variable) + "," + str(new_op.value)+"\n")
                    elif(new_op.op_type==False):
                        doc.write("read," + str(new_op.variable) + "," + str(new_op.value)+"\n")
                    else:
                        print("Error in file Writing!")
        doc.close()
        print(output_path+"hist_"+str(h)+".txt"+" succeeded.")


In [11]:
def write_result(result_single_history,file):
    '''
        result_single_history is a three dimensional list
        file is the output path
    '''
    f=open(file,"w")
    for n_clients in range(0,len(result_single_history)):
        for n_trans in range(0,len(result_single_history[0])):
            for n_ops in range(0,len(result_single_history[0][0])):
                f.write(result_single_history[n_clients][n_trans][n_ops]+'\n')
    f.close()
    print(file+ ' is completed.')

In [12]:
class MyThread(threading.Thread):
    def __init__(self, func, args, name=''):
        threading.Thread.__init__(self)
        self.name = name
        self.func = func
        self.args = args
        self.result = self.func(*self.args)
 
    def get_result(self):
        try:
            return self.result
        except Exception:
            return None

In [37]:
import docker
client = docker.from_env()
container = client.containers.get('galera_galera_3')
container.start()

cmd = 'mysql -uroot --execute="CREATE DATABASE IF NOT EXISTS galera;" '
container.exec_run(cmd)
cmd1 = 'mysql -uroot --execute="DROP TABLE IF EXISTS galera.variables;" '
cmd2 = 'mysql -uroot --execute="CREATE TABLE IF NOT EXISTS galera.variables (var BIGINT(64) UNSIGNED NOT NULL PRIMARY KEY, val BIGINT(64) UNSIGNED NOT NULL);"'
cmd3 = 'mysql -uroot --execute="INSERT INTO galera.variables (var, val) values (1, 1);"'
cmd4 = 'mysql -uroot --execute="SELECT * FROM galera.variables;" '
# cmd1 ='mysql -uroot --execute="show databases;" '
# res1 = container.exec_run(cmd1)
# container.exec_run(cmd1)
container.exec_run(cmd2)
container.exec_run(cmd3)
res =  container.exec_run(cmd4)
print(res.output)





b'ERROR 2002 (HY000): Can\'t connect to local MySQL server through socket \'/var/lib/mysql/mysql.sock\' (2 "No such file or directory")\n'


In [ ]:
'''
    思路：
    确定参数之后，先Creat Container,database，table，创建所有variable，且值都为0。
    参考dbcop
    相对应，exec_history用来multithread,每一个client是一个thread。
    具体做的内容是exec_session，exec_session内使用mysql pool？
'''

In [13]:
'''
    Use docker compose-up to create containers depending on how much clients;
'''
def create_containers(n_nodes):
    os.system('cd ../../dbcop/docker/galera')
    os.system('docker-compose up -d --scale galera0=1 --scale galera=' + str(n_nodes-1))
    # start all containers
    for i in range (0,n_nodes):
        client = docker.from_env()
        if(i==0):
            container = client.containers.get('galera0')
        else:
            container = client.containers.get('galera_galera_' + str(i))
        container.start()

In [14]:
def drop_tables(n_nodes):
    for i in range (0,n_nodes):
        client = docker.from_env()
        if(i==0):
            container = client.containers.get('galera0')
        else:
            container = client.containers.get('galera_galera_' + str(i))
        cmd = 'mysql -uroot --execute="DROP TABLE IF EXISTS galera.variables;" '
        container.exec_run(cmd)

In [15]:
def stop_containers(n_clients):
    for i in range (0,n_clients):
        client = docker.from_env()
        if(i==0):
            container = client.containers.get('galera0')
        else:
            container = client.containers.get('galera_galera_' + str(i))
        container.stop()

In [16]:
def create_variables(n_trans,n_variable):
    client = docker.from_env()
    container = client.containers.get('galera0')
    
    cmd1 = 'mysql -uroot --execute="CREATE DATABASE IF NOT EXISTS galera;" '
    cmd2 = 'mysql -uroot --execute="DROP TABLE IF EXISTS galera.variables;" '
    cmd3 = 'mysql -uroot --execute="CREATE TABLE IF NOT EXISTS galera.variables (var BIGINT(64) UNSIGNED NOT NULL PRIMARY KEY, val BIGINT(64) UNSIGNED NOT NULL);"'
    container.exec_run(cmd1)
    container.exec_run(cmd2)
    container.exec_run(cmd3)

    for j in range(0,n_trans):
        for i in range(0,n_variable):
            cmd4 = 'mysql -uroot --execute="INSERT INTO galera.variables (var, val) values ({0}, 0);"'.format(str(i))
            container.exec_run(cmd4)
    

            

In [17]:
def generate_clients(hist_file,n_clients,n_trans,n_ops):
    #input a single history file, contains n transactions.
    #Output a list of transaction,each transaction is a list of operation
    fo = open(hist_file, "r")
    print ("文件名为: ", fo.name)
    list_line = []
    for line in fo.readlines():                          #依次读取每行  
        line = line.strip()                             #去掉每行头尾空白,line is a list, contains all ops in hist file
        list_line.append(line)
    # 关闭文件
    fo.close()
    # print(list_line)
#     need a three dimension list: clients,trans,ops
    start = 0
    end = n_ops
    list_clients = []
    
    
    for i in range(0,n_clients):
        temp_trans = []
        for j in range(0,n_trans):
            temp_ops = list_line[start:end]
            temp_trans.append(temp_ops)
            start = start+n_ops
            end = end+n_ops
        list_clients.append(temp_trans)
        
    return list_clients

In [18]:
'''
    execute a history file in multiple threads(clients).
    history data is stored in a three dimensional list, list_of_ops.
    
'''
def exec_history(list_of_ops):
    n_clients = len(list_of_ops)
    n_trans = len(list_of_ops[0])
    n_ops = len(list_of_ops[0][0])
    thread_clients = []
#     each thread calls this: exec_session(list_of_ops[i_thread],i_result_ops), i_result_ops is a two dimensional list
    result_clients = []

    for i in range(0,n_clients):
        client = docker.from_env()
        new_client = MyThread(exec_session,(list_of_ops[i],client, i,), exec_session.__name__)
        thread_clients.append(new_client)
    for i in range(0,n_clients):
        thread_clients[i].start()
    for i in range(0,n_clients):
        thread_clients[i].join()
        
    for i in range(0,n_clients):
        temp_result = thread_clients[i].get_result()
#         print(temp_result)
        result_clients.append(temp_result)
    
    return result_clients

In [22]:
'''
    list_of_ops is a 2 dimensional list, contains trans and ops.
    each client is a thread.
'''
def exec_session(list_of_ops,client, client_id):
    result_ops = []
    n_trans = len(list_of_ops)
    op_num = 0
    if(client_id==0):
        container = client.containers.get('galera0')
    else:
        node_num = random_pick([1,2,3],[0.4,0.3,0.3])
        container = client.containers.get('galera_galera_' + str(node_num))
    
    for i in range(0,n_trans):
        
        temp_tx_op = []
        for j in range(0,len(list_of_ops[i])):
#             print('now dealing with : '+list_of_ops[i][j])
            op = str.split(list_of_ops[i][j],',',3)
            # create a new variable
            key = int(op[1])

            
            if(op[0]=='write'):
                # if op is a write, then set value to this key
                val = int(op[2])
                cmd = 'mysql -uroot --execute="UPDATE galera.variables SET val={0} WHERE var={1}"'.format(str(val),str(key))
                container.exec_run(cmd)
                
                cmd2 = 'mysql -uroot --execute="SELECT val FROM galera.variables WHERE var={};" '.format(str(key))
                res = container.exec_run(cmd2)

                # record the op, dealing with res
                record_key = key
#                 print('write key: '+str(record_key))

                record_val = res.output.decode("utf-8").replace("\n", "")[3:]
#                 print('wirte key with value: '+str(record_val))
                single_op = 'w(' +  str(record_key) +',' + str(record_val) +','+ str(client_id) +','+str(i) +','+ str(op_num)+')'
                temp_tx_op.append(single_op)
                op_num = op_num +1
            elif(op[0]=='read'):
                cmd2 = 'mysql -uroot --execute="SELECT val FROM galera.variables WHERE var={};" '.format(str(key))
                res = container.exec_run(cmd2)
                # print( res[0].value())
                record_key = key
#                 print('read key: '+str(record_key))
                record_val = res.output.decode("utf-8").replace("\n", "")[3:]
#                 print('read key with value: '+str(record_val))
                
                single_op = 'r(' +  str(record_key) +',' + str(record_val) +','+ str(client_id) +','+str(i) +','+ str(op_num)+')'
                temp_tx_op.append(single_op)
                op_num = op_num +1
            else:
                print("unknown wrong type op.")
        result_ops.append(temp_tx_op)
    
    return result_ops

In [23]:
def main():
    client_num = [20]
    transaction_num = [100]
    wr_profile = [55]
    key_num = [100]
    
    for c in client_num:
        for t in transaction_num:
            for wr in wr_profile:
                for k in key_num:
                    path = '../DATATEST/galera/client' + str(c) + '/transaction' + str(t) + '/wr' + str(wr) + '/key' +str(k) +'/zipf/'
                    
#                     zipf_generator(path,100,c,t,1,k,wr)
                    for i in range(1,100):
#                         create_containers(c)
                        generate_file = path + 'hist_'+str(i)+'.txt'
                        result_file = path + 'result_'+str(i)+'.txt'

                        create_variables(1,k)
                        list_of_ops = generate_clients(generate_file,c,t,1)
                        result_single_history = exec_history(list_of_ops)
                        write_result(result_single_history,result_file)
                        drop_tables(4)
#                         stop_containers(c)
                        
                        
                        

if __name__ == '__main__':
    try:
        main()
        print('DONE!')
    except Exception as e:
        print('Error: {}'.format(e))


文件名为:  ../DATATEST/galera/client20/transaction100/wr55/key100/zipf/hist_1.txt
../DATATEST/galera/client20/transaction100/wr55/key100/zipf/result_1.txt is completed.
文件名为:  ../DATATEST/galera/client20/transaction100/wr55/key100/zipf/hist_2.txt
../DATATEST/galera/client20/transaction100/wr55/key100/zipf/result_2.txt is completed.
文件名为:  ../DATATEST/galera/client20/transaction100/wr55/key100/zipf/hist_3.txt
../DATATEST/galera/client20/transaction100/wr55/key100/zipf/result_3.txt is completed.
文件名为:  ../DATATEST/galera/client20/transaction100/wr55/key100/zipf/hist_4.txt
../DATATEST/galera/client20/transaction100/wr55/key100/zipf/result_4.txt is completed.
文件名为:  ../DATATEST/galera/client20/transaction100/wr55/key100/zipf/hist_5.txt
../DATATEST/galera/client20/transaction100/wr55/key100/zipf/result_5.txt is completed.
文件名为:  ../DATATEST/galera/client20/transaction100/wr55/key100/zipf/hist_6.txt
../DATATEST/galera/client20/transaction100/wr55/key100/zipf/result_6.txt is completed.
文件名为:  ../

../DATATEST/galera/client20/transaction100/wr55/key100/zipf/result_50.txt is completed.
文件名为:  ../DATATEST/galera/client20/transaction100/wr55/key100/zipf/hist_51.txt
../DATATEST/galera/client20/transaction100/wr55/key100/zipf/result_51.txt is completed.
文件名为:  ../DATATEST/galera/client20/transaction100/wr55/key100/zipf/hist_52.txt
../DATATEST/galera/client20/transaction100/wr55/key100/zipf/result_52.txt is completed.
文件名为:  ../DATATEST/galera/client20/transaction100/wr55/key100/zipf/hist_53.txt
../DATATEST/galera/client20/transaction100/wr55/key100/zipf/result_53.txt is completed.
文件名为:  ../DATATEST/galera/client20/transaction100/wr55/key100/zipf/hist_54.txt
../DATATEST/galera/client20/transaction100/wr55/key100/zipf/result_54.txt is completed.
文件名为:  ../DATATEST/galera/client20/transaction100/wr55/key100/zipf/hist_55.txt
../DATATEST/galera/client20/transaction100/wr55/key100/zipf/result_55.txt is completed.
文件名为:  ../DATATEST/galera/client20/transaction100/wr55/key100/zipf/hist_56.tx

KeyboardInterrupt: 